In [ ]:
#Bayesian Average

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ratingavg = [4,4.3,4.25,4.5,4.6,4.7,5]
ratingamount = [10000,345,9500,4234,5524,2000,100]
dfzip = list(zip(ratingavg, ratingamount))
df = pd.DataFrame(dfzip, columns = ['avg rating', 'num of ratings'])

In [ ]:
ratingavg = [5,4.8,4.6]
ratingamount = [10,100,1000]
dfzip = list(zip(ratingavg, ratingamount))
df = pd.DataFrame(dfzip, columns = ['average rating', 'num of ratings'])

In [ ]:
df = pd.read_csv('moviesetwratings.csv')

In [ ]:
df

,Unnamed: 0,titles,summaries,genre1,genre2,genre3,average rating,num of ratings
0,0,Doctor Strange in the Multiverse of Madness,Doctor Strange teams up with a mysterious teen...,Action,Adventure,Fantasy,7.4,165258
1,1,The Northman,From visionary director Robert Eggers comes Th...,Action,Adventure,Drama,7.4,87350
2,2,The Batman,When a sadistic serial killer begins murdering...,Action,Crime,Drama,8.0,478796
3,3,The Lost City,A reclusive romance novelist on a book tour wi...,Action,Adventure,Comedy,6.2,44764
4,4,Everything Everywhere All at Once,An aging Chinese immigrant is swept up in an i...,Action,Adventure,Comedy,8.6,63228
...,...,...,...,...,...,...,...,...
20849,20849,City of Mice,Probably the most famous puppet show for child...,Adventure,Comedy,Family,7.3,1429
20850,20850,Tengamos la fiesta en paz,Add a Plot,Comedy,Family,Musical,8.4,37
20851,20851,Devadasu,"Adapted from a popular Bengali novel, Devadasu...",Drama,Musical,Romance,7.7,276
20852,20852,Carnival in Moscow,"A new chief of a ""Culture House"" is planning t...",Comedy,Drama,Musical,7.5,2669


In [ ]:
c = df['num of ratings'].quantile(.25) #100


In [ ]:
avg

4.8

In [ ]:
#function using the equation from https://www.algolia.com/doc/guides/solutions/ecommerce/relevance-optimization/tutorials/bayesian-average/
def bayesAvg(df,rating,votes):

  #(ProductRatingAverage * ProductRatingCount) + (25 percentile * (avg of all ratings)) / (ProductRatingAverage + 25 percentile)

  df['num'] = df[rating] * df[votes]
  totalavg = df['num'].sum()
  totalavg = totalavg / df[votes].sum()

  c = df[votes].quantile(.25)

  return (df[rating] * df[votes] + c * totalavg ) / (df[votes] + c)




In [ ]:
df['bayesian average'] = bayesAvg(df,'average rating','num of ratings')
df = df.drop(['num'], axis=1)
df.sort_values(by=['bayesian average'], ascending=False)

,Unnamed: 0,titles,summaries,genre1,genre2,genre3,average rating,num of ratings,bayesian average
56,56,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Drama,,,9.3,2589409,9.299409
49,49,The Godfather,The aging patriarch of an organized crime dyna...,Crime,Drama,,9.2,1784696,9.199182
33,33,The Dark Knight,When the menace known as the Joker wreaks havo...,Action,Crime,Drama,9.0,2560072,8.999485
650,650,The Lord of the Rings: The Return of the King,Gandalf and Aragorn lead the World of Men agai...,Action,Adventure,Drama,9.0,1779194,8.999258
326,326,Schindler's List,"In German-occupied Poland during World War II,...",Biography,Drama,History,9.0,1317658,8.998999
...,...,...,...,...,...,...,...,...,...
3943,3943,Foodfight!,The evil Brand X joins a supermarket that beco...,Animation,Action,Adventure,1.3,10836,1.657270
9317,9317,Superbabies: Baby Geniuses 2,A group of smart-talking toddlers find themsel...,Comedy,Family,Sci-Fi,1.5,30985,1.625665
9710,9710,The Cost of Deception,"When a young, ambitious market researcher find...",Crime,Drama,History,1.5,37863,1.603256
10666,10666,Kirk Cameron's Saving Christmas,His annual Christmas party faltering thanks to...,Comedy,Family,,1.3,15848,1.549107


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
pd.set_option('display.max_colwidth',9999)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
ps = PorterStemmer()

In [ ]:
#porter stem

porterstem = df.copy()

for s in porterstem['summaries']:
  x = 0
  stemmed = word_tokenize(s)
  for y in stemmed:
    stemmed[x] = ps.stem(y)
    x = x + 1
  string = " ".join(stemmed)
  porterstem = porterstem.replace([s],string)


In [ ]:
porterstem.sort_values(by=['bayesian average'], ascending=False)
newbow = porterstem.copy()

In [ ]:
CountSum = CountVectorizer(ngram_range=(1,1),stop_words='english',min_df=7)
CountSumData = CountSum.fit_transform(porterstem['summaries'])

In [ ]:
bow = pd.DataFrame(CountSumData.toarray(),columns=CountSum.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
newdf = pd.concat([newbow,bow],axis=1)

In [ ]:
newdf

,Unnamed: 0,titles,summaries,genre1,genre2,genre3,average rating,num of ratings,bayesian average,000,...,yugoslav,yugoslavia,yukon,yuma,zani,zealand,zero,zombi,zone,zoo
0,0,Doctor Strange in the Multiverse of Madness,"doctor strang team up with a mysteri teenag girl from hi dream who can travel across multivers , to battl multipl threat , includ other-univers version of himself , which threaten to wipe out million across the multivers . they seek help from wanda the scarlet witch , wong and other .",Action,Adventure,Fantasy,7.4,165258,7.398863,0,...,0,0,0,0,0,0,0,0,0,0
1,1,The Northman,"from visionari director robert egger come the northman , an action-fil epic that follow a young vike princ on hi quest to aveng hi father 's murder .",Action,Adventure,Drama,7.4,87350,7.397857,0,...,0,0,0,0,0,0,0,0,0,0
2,2,The Batman,"when a sadist serial killer begin murder key polit figur in gotham , batman is forc to investig the citi 's hidden corrupt and question hi famili 's involv .",Action,Crime,Drama,8.0,478796,7.998722,0,...,0,0,0,0,0,0,0,0,0,0
3,3,The Lost City,A reclus romanc novelist on a book tour with her cover model get swept up in a kidnap attempt that land them both in a cutthroat jungl adventur .,Action,Adventure,Comedy,6.2,44764,6.214508,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Everything Everywhere All at Once,"An age chines immigr is swept up in an insan adventur , where she alon can save the world by explor other univers connect with the live she could have led .",Action,Adventure,Comedy,8.6,63228,8.583778,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20849,20849,City of Mice,probabl the most famou puppet show for children and all age . mice who have to leav their school and immigr becaus of a wild cat who want to eat them up . eventu the mice defeat the cat and go back to their home .,Adventure,Comedy,Family,7.3,1429,7.244743,0,...,0,0,0,0,0,0,0,0,0,0
20850,20850,Tengamos la fiesta en paz,add a plot,Comedy,Family,Musical,8.4,37,7.196063,0,...,0,0,0,0,0,0,0,0,0,0
20851,20851,Devadasu,"adapt from a popular bengali novel , devadasu narrat the stori of a rich guy and poor girl separ by class system of india . parvati is forc marri off to a middle-ag rich man and devadasu becom a drunkard .",Drama,Musical,Romance,7.7,276,7.292239,0,...,0,0,0,0,0,0,0,0,0,0
20852,20852,Carnival in Moscow,A new chief of a `` cultur hous '' is plan to hold a terribl bore new year concert . A group of young amateur actor are do their best to liven up the concert .,Comedy,Drama,Musical,7.5,2669,7.423155,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def changeones(newdf):
  newdf.replace("old string", "new string")
    

In [ ]:
newdf.sort_values(by=['bayesian average'], ascending=False)
newdf = changeones(newdf)
print(newdf['zoo'].dtypes)

int64
